In [2]:
from ansys.dpf import post
from ansys.dpf import core as dpf
from utils.ansys_utils import run_cfile_ls_prepost
from utils.ls_prepost_utils import *
import numpy as np

In [3]:
# root = os.path.dirname(os.path.abspath(__file__))
root = os.getcwd()
d3plot_path = os.path.join(root, "output", "d3plot")
keyword_path = os.path.join(root, "output", "ball_plate.k")
cfile_path = os.path.join(root, "cfile", "element_mass_all.cfile")
msg_path = os.path.join(root, "cfile", "lspost.msg")

## Load simulation data

In [4]:
# Load server
server = dpf.start_local_server(ansys_path=r"C:\Program Files\ANSYS Inc\v242", as_global=True)

In [5]:
model = load_model_metadata(d3plot_path)
mesh = load_mesh(d3plot_path)

### Get information from mesh

In [6]:
# Read mesh 
element_to_nodes = build_element_connectivity(mesh)
element_to_material = build_element_material_map(mesh)

#### Calculate mass of elements
# Create ls-dyna cfile 
write_mass_cfile(cfile_path, keyword_path, element_to_nodes)
# Run cfile 
# run_cfile_ls_prepost(cfile_path)
# Get mass elements
element_masses = read_element_mass_map(msg_path)
node_mass, missing_elements = compute_node_masses(element_masses, element_to_nodes)

print(f"Elements: {len(element_to_nodes)}")
print(f"Materials: {len(element_to_material)}")
print(f"Element masses parsed: {len(element_masses)}")
print(f"Node masses computed: {len(node_mass)}")
if missing_elements:
    print(f"Missing element masses: {len(missing_elements)}")

Elements: 1768
Materials: 1768
Element masses parsed: 1768
Node masses computed: 1940


In [7]:
node_mass_data_gnn = np.array(list(node_mass.values()))

In [8]:
# Build edges for GNN
    

def build_edges_gnn_from_mesh(mesh):
    """
    Build directed edges (2 per undirected mesh edge) for a GNN from a mesh.

    Returns
    -------
    edges_gnn : np.ndarray, shape (E, 2), dtype=int
        Directed edges in zero-based node indexing.
    edges_attr : np.ndarray, shape (E,)
        Edge attribute per directed edge (here: material id/value per element).
    """
    edges_list = []
    attr_list = []

    for element_index, element in enumerate(mesh.elements):
        etype = str(element.type).split(".")[-1]  # e.g. element_types.Hex8 -> "Hex8"
        node_ids = element.node_ids

        if etype not in REAL_EDGES:
            continue

        # element-level attribute (material for this element)
        mat = mesh.materials.array[element_index]

        for i, j in REAL_EDGES[etype]:
            n1 = int(node_ids[i]) - 1  # zero-based
            n2 = int(node_ids[j]) - 1  # zero-based

            # add both directions
            edges_list.append((n1, n2))
            attr_list.append(mat)

            edges_list.append((n2, n1))
            attr_list.append(mat)

    if not edges_list:
        return np.zeros((0, 2), dtype=int), np.zeros((0,), dtype=np.asarray(mesh.materials.array).dtype)

    edges_gnn = np.asarray(edges_list, dtype=int)
    edges_attr = np.asarray(attr_list)

    # Remove duplicate edges AND keep edges_attr aligned:
    # unique rows with indices of first occurrence
    _, unique_idx = np.unique(edges_gnn, axis=0, return_index=True)
    unique_idx = np.sort(unique_idx)  # keep a stable order

    edges_gnn = edges_gnn[unique_idx]
    edges_attr = edges_attr[unique_idx]

    return edges_gnn, edges_attr

In [9]:
edges_gnn, edges_attr = build_edges_gnn_from_mesh(mesh)

### Get Node information

In [10]:
# Get node features 
coordinates_fc:dpf.FieldsContainer = model.results.coordinates.on_all_time_freqs.eval() # mm unit
acceleration_fc:dpf.FieldsContainer = model.results.acceleration.on_all_time_freqs.eval() # mm/ms^2 unit
velocity_fc:dpf.FieldsContainer = model.results.velocity.on_all_time_freqs.eval() # mm/ms unit
displacement_fc:dpf.FieldsContainer = model.results.displacement.on_all_time_freqs.eval() # mm unit
kinetic_energy = model.results.global_kinetic_energy.eval()
internal_energy = model.results.global_internal_energy.eval()
total_energy = model.results.global_total_energy.eval() 
# stress_field_container:dpf.FieldsContainer = model.results.stress.on_all_time_freqs.eval() # MPa unit
time = np.array(model.metadata.time_freq_support.time_frequencies.data) # ms unit
# delta_t = time[1:] - time[:-1]

In [11]:
print(kinetic_energy)

DPF  Fields Container
  with 1 field(s)
  defined on labels: 

  with:
  - field 0 {} with TimeFreq_steps location, 1 components and 200 entities.



In [12]:
all_features = []   # list of T tensors
predict_features = []
delta_T = []
total_kinetic_energy = []
total_internal_energy = []
num_series = 3
num_steps = len(time)

# Step1: Extract node features for entire node at every time step
node_feat_series = [] #
for i in range(num_steps): # Start from num_series-1 to T-1
    coor = np.asarray(coordinates_fc[i].data)
    acc  = np.asarray(acceleration_fc[i].data)
    vel  = np.asarray(velocity_fc[i].data)
    disp = np.asarray(displacement_fc[i].data)
    node_feat_series.append(np.concatenate([coor, acc, vel, disp], axis=1)) # (N, 12)
node_feat_series = np.stack(node_feat_series, axis=2) # (N, 12, T) 

for t in range(num_series - 1, num_steps - 1):

    # Extract node features for entire node at time t with history
    node_feat = node_feat_series[:, :, t - num_series + 1 : t + 1]  # (N, 12, num_series)
    all_features.append(node_feat)

    # Predict residual
    x_t = node_feat_series[:,:,t]
    x_t_1 = node_feat_series[:,:,t+1]
    y_residual = x_t_1 - x_t

    # Delta time
    delta_t = time[t+1] - time[t]
    delta_T.append(delta_t)
    y_change = y_residual / delta_t
    predict_features.append(y_change)

    # Store total energies
    total_kinetic_energy.append(kinetic_energy[0].data[t])
    total_internal_energy.append(internal_energy[0].data[t])

In [17]:
total_kinetic_energy[0]

np.float64(1.9771696329116821)

In [ ]:
data_path = os.path.join(root, "data", "ball_plate_gnn_data.npz")

solids = {eid:n for eid,n in element_to_nodes.items() if len(n)==8}
shells = {eid:n for eid,n in element_to_nodes.items() if len(n)==4}

np.savez_compressed(
    data_path,
    X_list=np.array(all_features),  # (num_samples, N, 12, num_series)
    Y_list=np.array(predict_features),  # (num_samples, N, 12)
    node_mass=node_mass_data_gnn,          # (N,)
    Delta_t=np.array(delta_T),
    element_id_solids=np.array(list(solids.keys())),  # (num_elements,)
    element_id_shells=np.array(list(shells.keys())),  # (num_elements,)
    element_to_nodes_solids=np.array(list(solids.values())),  # (num_elements, nodes_per_element)
    element_to_nodes_shells=np.array(list(shells.values())),  # (num_elements, nodes_per_element)
    element_materials=np.array(list(element_to_material.values())),  # (num_elements,)
    edge_index=edges_gnn,                  # (E, 2)
    edge_attr=edges_attr,                  # (E,)
    total_internal_energy=total_internal_energy,    # (num_series,)
    total_kinetic_energy=total_kinetic_energy,    # (num_series,)
)

In [21]:
solids

{257: [290, 339, 346, 297, 291, 340, 347, 298],
 258: [291, 340, 347, 298, 292, 341, 348, 299],
 259: [292, 341, 348, 299, 293, 342, 349, 300],
 260: [293, 342, 349, 300, 294, 343, 350, 301],
 261: [294, 343, 350, 301, 295, 344, 351, 302],
 262: [295, 344, 351, 302, 296, 345, 352, 303],
 263: [297, 346, 353, 304, 298, 347, 354, 305],
 264: [298, 347, 354, 305, 299, 348, 355, 306],
 265: [299, 348, 355, 306, 300, 349, 356, 307],
 266: [300, 349, 356, 307, 301, 350, 357, 308],
 267: [301, 350, 357, 308, 302, 351, 358, 309],
 268: [302, 351, 358, 309, 303, 352, 359, 310],
 269: [304, 353, 360, 311, 305, 354, 361, 312],
 270: [305, 354, 361, 312, 306, 355, 362, 313],
 271: [306, 355, 362, 313, 307, 356, 363, 314],
 272: [307, 356, 363, 314, 308, 357, 364, 315],
 273: [308, 357, 364, 315, 309, 358, 365, 316],
 274: [309, 358, 365, 316, 310, 359, 366, 317],
 275: [311, 360, 367, 318, 312, 361, 368, 319],
 276: [312, 361, 368, 319, 313, 362, 369, 320],
 277: [313, 362, 369, 320, 314, 363, 370